In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init() 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-08-31 00:00:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [7]:
# Filter the data and create a new DataFrame to retrieve all the rows where total_votes count is equal to or greater than 20

high_votes_df = vine_df[vine_df["total_votes"] >= 20]
high_votes_df.sort("total_votes", ascending=True).show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VAO76275GRHU|          5|           20|         20|   N|                Y|
|R37NBD652CWUD3|          5|           16|         20|   N|                Y|
| RKWUQZYJBZX7K|          5|           20|         20|   N|                Y|
|R2WYZJIBXGPZ5N|          1|           16|         20|   N|                Y|
| RW8Q6B2KN43LO|          5|           20|         20|   N|                N|
| RW47OAPTH46S2|          1|           19|         20|   N|                Y|
| RNKK5KFNZBXGI|          2|           19|         20|   N|                Y|
| RAHM9DB2ENJR8|          5|           17|         20|   N|                N|
|R1B0AR7UUBIZ8P|          5|           20|         20|   N|                N|
|R33BS2KAL7W37Z|          5|           20|         20|   N|     

In [8]:
# Create dataframe to retrieve all rows where the percent of helpful votes is greater than or equal to 50%
most_helpful_df = high_votes_df.withColumn('percent_helpful', (high_votes_df['helpful_votes'] / high_votes_df['total_votes']) * 100)
most_helpful_df = most_helpful_df[most_helpful_df["percent_helpful"] >= 50]
most_helpful_df.sort("percent_helpful", ascending=True).show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|R2FOR0B8GANVAF|          1|           18|         36|   N|                N|           50.0|
|R37158XJ9QX9KK|          2|           11|         22|   N|                Y|           50.0|
|R2OQACGJNYX822|          1|           10|         20|   N|                Y|           50.0|
| RWBLD0O1LA0YP|          1|           10|         20|   N|                Y|           50.0|
|R2X2Z4U33JN75Q|          3|           13|         26|   N|                N|           50.0|
|R2WUDEIKR0FEUA|          1|           10|         20|   N|                N|           50.0|
| R1JDT6X3HJCB9|          4|           13|         26|   N|                N|           50.0|
|R21LFR7KRZE10J|          5|           12|         24|   N| 

In [9]:
# Create dataframe that retrieves all the rows where a review was written as part of the Vine proram
vine_program_df = most_helpful_df[most_helpful_df['vine'] == 'Y']
vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|             90.0|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|            100.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|92.85714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|96.66666666666667|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|            100.0|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|96.15384615384616|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|             96.0|
|R25FI3J7WWOYXM|          3|           5

In [10]:
# Create dataframe that retrieves all the rows where a review was NOT part of the Vine proram
not_vine_df = most_helpful_df[most_helpful_df['vine'] == 'N']
not_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|87.09677419354838|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|            100.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|93.54838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|83.33333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|96.42857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           96.875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|83.72093023255815|
|R3A1VQ3SQDXEJ3|          5|           2

In [11]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5 start reviews for the two types of review
columns = ["program_status", "total_reviews", "five_star_reviews", "percent_five_star"]
vine_total = vine_program_df.count()
not_vine_total = not_vine_df.count()
vine_5_star = (vine_program_df[vine_program_df["star_rating"] == 5]).count()
not_vine_5_star = (not_vine_df[not_vine_df["star_rating"] == 5]).count()
vine_percent_5 = (vine_5_star / vine_total) * 100
not_vine_percent_5 = (not_vine_5_star / not_vine_total) * 100
data = [["Vine Program", vine_total, vine_5_star, vine_percent_5],
        ["Not in Vine Program", not_vine_total, not_vine_5_star, not_vine_percent_5]]
summary_df = spark.createDataFrame(data, columns)
summary_df.show()

+-------------------+-------------+-----------------+------------------+
|     program_status|total_reviews|five_star_reviews| percent_five_star|
+-------------------+-------------+-----------------+------------------+
|       Vine Program|          170|               65| 38.23529411764706|
|Not in Vine Program|        37840|            20612|54.471458773784356|
+-------------------+-------------+-----------------+------------------+

